In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/owid-covid-data.csv
/kaggle/input/US_counties.csv


# A structured Step-by-Step Process

1. Define Problem.
2. Summarize Data.
3. Prepare Data.
4. Evaluate Algorithm
5. Improve results
6. Present Results.


## 1. Problem definition

The purpose of this model is to assess risk of COVID-19 exposure based on places according to number of cases and deaths. The input data to the model would be place (zip-code), and the output would be a risk score measure based on the input attributes calculated within the model.

This dataset was collected by [Minh Son Nguyen](emailto://son.nguyen.ohiou@gmail.com)

In [ ]:
US_counties = pd.read_csv('../input/US_counties.csv')

In [10]:
owid_covid = pd.read_csv('../input/owid-covid-data.csv')

## 2. Summary statistics

Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:

*  Dimensions of the dataset.
*  Peek at the data itself.
*  Statistical summary of all attributes.
*  Breakdown of the data by the class variable.

Starting by the first dataset: **US_counties**

*  Dimensions of the dataset.

In [9]:
#shape 
print("US_counties dimention is: " + str(US_counties.shape))

US_counties dimention is: (310256, 6)


The US_counties has 310256 observations and 6 features

*  Peek at the data itself.

In [5]:
#head
display(US_counties.head())

,date,county,state,fips,cases,deaths
0,04/13/20,Motley,Texas,48345.0,1,0
1,04/14/20,Motley,Texas,48345.0,1,0
2,04/15/20,Motley,Texas,48345.0,1,0
3,04/16/20,Motley,Texas,48345.0,1,0
4,04/17/20,Motley,Texas,48345.0,1,0


In [8]:
print("Types of the data: ", str(US_counties.dtypes))

Types of the data:  date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object


## Attributes:
The US_counties dataset has six(6 attributes) as the followings:
 - date: date of the reported cases.
 - county: The US county.
 - state: The US state.
 - fips: *Federal Information Processing Standard Publication*, identify each US county to join with other tables.
 - cases: Number of cases reported per day.
 - deaths: Number of deaths reported per day.

* Statistical Summary

In [13]:
# description
pd.set_option('precision', 2)
print(US_counties.describe())

            fips      cases     deaths
count  307024.00  310256.00  310256.00
mean    30152.99     498.09      26.57
std     15318.26    4122.34     358.55
min      1001.00       0.00       0.00
25%     18101.00       6.00       0.00
50%     29081.00      30.00       1.00
75%     45053.00     144.00       4.00
max     56045.00  222444.00   22696.00


In [14]:
print(US_counties.groupby('state').size())

state
Alabama                      7154
Alaska                       1852
Arizona                      1750
Arkansas                     7808
California                   6613
Colorado                     6481
Connecticut                  1025
Delaware                      435
District of Columbia          124
Florida                      7530
Georgia                     17049
Guam                          116
Hawaii                        498
Idaho                        3602
Illinois                    10164
Indiana                      9878
Iowa                         9673
Kansas                       8403
Kentucky                    11692
Louisiana                    7093
Maine                        1813
Maryland                     2795
Massachusetts                1804
Michigan                     8586
Minnesota                    8587
Mississippi                  8817
Missouri                    10741
Montana                      3279
Nebraska                     6604
Nevada  

As demonestrated above, each state has  the same number of instances

In [ ]:
pd.set_option('display.max_columns', None)
display(owid_covid.head())